In [ ]:
# --- Install dependencies (run these in Colab, if needed) ---
!apt-get install -y ffmpeg
!git clone https://github.com/tyiannak/pyAudioAnalysis.git
%cd pyAudioAnalysis
!pip install -r requirements.txt
!python3 setup.py install
%cd ..

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Cloning into 'pyAudioAnalysis'...
remote: Enumerating objects: 3044, done.
remote: Counting objects: 100% (517/517), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 3044 (delta 430), reused 396 (delta 396), pack-reused 2527 (from 1)
Receiving objects: 100% (3044/3044), 167.66 MiB | 20.36 MiB/s, done.
Resolving deltas: 100% (1818/1818), done.
/content/pyAudioAnalysis
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 11.4 MB/s eta 0:00:00
running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        **********************************************************

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
# --- Import Libraries ---
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import ShortTermFeatures
import librosa
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

# --- Load audio file ---
file_path = "harvard.wav"
y, sr = librosa.load(file_path, sr=16000)

# --- Feature: Loudness (RMS Energy) ---
rms = librosa.feature.rms(y=y)[0]
avg_loudness = np.mean(rms)
energy_std = np.std(rms)

# --- Feature: Pitch (Fundamental Frequency) ---
pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
pitches = pitches[magnitudes > np.median(magnitudes)]  # Filter low energy
mean_pitch = np.mean(pitches) if len(pitches) > 0 else 0
pitch_std = np.std(pitches) if len(pitches) > 0 else 0

# --- Feature: Speaking Rate (Tempo Proxy) ---
onset_env = librosa.onset.onset_strength(y=y, sr=sr)
tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]

# --- Feature: Silence Ratio ---
frame_length = 1024
hop_length = 512
energy = np.array([
    np.sum(np.abs(y[i:i+frame_length]**2))
    for i in range(0, len(y), hop_length)
])
threshold = np.percentile(energy, 10)
silent_frames = energy < threshold
silence_ratio = np.sum(silent_frames) / len(energy)

# --- Additional Features using pyAudioAnalysis ---
[fs, x] = audioBasicIO.read_audio_file(file_path)
x = audioBasicIO.stereo_to_mono(x)
F, f_names = ShortTermFeatures.feature_extraction(x, fs, 0.050*fs, 0.025*fs)

# Extracted Features
zcr = np.mean(F[0])  # Zero Crossing Rate
spectral_centroid = np.mean(F[4])  # Spectral Centroid

# --- Print All Features ---
print("🎤 Audio-Based Language Proficiency Features\n")

print(f"🔊 Average Loudness (RMS):           {avg_loudness:.4f}")
print(f"⚡ Energy Variation (RMS STD):       {energy_std:.4f}")
print(f"🎵 Mean Pitch (Hz):                  {mean_pitch:.2f}")
print(f"🎶 Pitch Variation (STD):            {pitch_std:.2f}")
print(f"🗣️ Speaking Rate (Tempo):            {tempo:.2f} BPM")
print(f"🤫 Silence Ratio (low energy):       {silence_ratio:.2%}")
print(f"📈 Zero Crossing Rate (ZCR):         {zcr:.4f}")
print(f"📍 Spectral Centroid (mean):         {spectral_centroid:.2f}")


<ipython-input-1-840adb8f8f15>:26: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]
/usr/local/lib/python3.11/dist-packages/pyAudioAnalysis-0.3.14-py3.11.egg/pyAudioAnalysis/audioBasicIO.py:99: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


🎤 Audio-Based Language Proficiency Features

🔊 Average Loudness (RMS):           0.0361
⚡ Energy Variation (RMS STD):       0.0428
🎵 Mean Pitch (Hz):                  1493.91
🎶 Pitch Variation (STD):            1137.66
🗣️ Speaking Rate (Tempo):            144.23 BPM
🤫 Silence Ratio (low energy):       10.10%
📈 Zero Crossing Rate (ZCR):         0.0687
📍 Spectral Centroid (mean):         0.19
